<a href="https://colab.research.google.com/github/pratapponnam/ProstateDetection/blob/main/nnunetv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nnunet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
pip install nnunetv2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 47.6 MB/s eta 0:00:00
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Created wheel for nnunetv2: filename=nnunetv2-2.6.0-py3-none-any.whl size=276978 sha256=56f8ebeb280b6bfc8c3932bda074852643e9921d0810de3b00e6d33df8fa8f5a
  Stored in directory: /root/.cache/pip/wheels/3d/0d/bb/f932cb0032d1aacc2f336de421448313d147db8eed671096e9
  Created wheel for ac

In [2]:
import os
import glob
import numpy as np
import nibabel as nib
import pydicom
import tensorflow as tf
import json
import pandas as pd
from tqdm import tqdm


In [4]:

# Paths
BASE_PATH = "/content/drive/MyDrive/"
DATASET_PATH = os.path.join(BASE_PATH, "Prostate")  # Already extracted
NIFTI_SAVE_PATH = os.path.join(BASE_PATH, "Prostate_NIfTI_Cache")  # Cache location
NNUNET_RAW_PATH = os.path.join(BASE_PATH, "nnUNet_raw/Task500_ProstateX")
IMAGES_TR_PATH = os.path.join(NNUNET_RAW_PATH, "imagesTr")
LABELS_TR_PATH = os.path.join(NNUNET_RAW_PATH, "labelsTr")

# Ensure directories exist
os.makedirs(IMAGES_TR_PATH, exist_ok=True)
os.makedirs(LABELS_TR_PATH, exist_ok=True)
os.makedirs(NIFTI_SAVE_PATH, exist_ok=True)

# Load CSV files
image_train_file = "/content/drive/MyDrive/ProstateX-Images-Train1.csv"
findings_train_file = "/content/drive/MyDrive/ProstateX-Findings-Train1.csv"

if not os.path.exists(image_train_file) or not os.path.exists(findings_train_file):
    print("[ERROR] One or both CSV files are missing. Check the paths.")
    exit(1)

# Load data
image_train_df = pd.read_csv(image_train_file)
findings_train_df = pd.read_csv(findings_train_file)

# Print column names to check
print("[INFO] ImageTrain Columns:", image_train_df.columns)
print("[INFO] FindingsTrain Columns:", findings_train_df.columns)

# Ensure 'ProxID' and 'fid' exist in both CSVs
required_columns = ["ProxID", "fid"]
for col in required_columns:
    if col not in image_train_df.columns or col not in findings_train_df.columns:
        print(f"[ERROR] Column '{col}' not found in both CSV files.")
        exit(1)

# Merge both datasets on 'ProxID' and 'fid'
merged_df = pd.merge(image_train_df, findings_train_df, on=["ProxID", "fid"], how="inner")

print(f"[INFO] Successfully merged! Total records: {len(merged_df)}")

# Correct column names
SERIES_DESC_COL = "DCMSerDescr"  # Column with Series Description
SERIES_NUM_COL = "DCMSerNum"  # Column with Series Number
LABEL_COL = "ClinSig"  # Column with Label

# Function to load or convert DICOM images
def load_or_convert_dicom_images(dicom_dir, merged_df, img_size=(128, 128)):
    images = []
    labels = []
    matched_records = 0

    dicom_files = glob.glob(os.path.join(dicom_dir, "**/*.dcm"), recursive=True)
    print(f"Found {len(dicom_files)} DICOM files.")

    if len(dicom_files) == 0:
        print("[ERROR] No DICOM files found. Check the directory path.")
        return [], []

    for idx, row in merged_df.iterrows():
        series_desc = row[SERIES_DESC_COL]
        series_num = row[SERIES_NUM_COL]
        label = row[LABEL_COL]

        nifti_file_path = os.path.join(NIFTI_SAVE_PATH, f"prostate_{idx:04d}.nii.gz")

        # If NIfTI already exists, use cached version
        if os.path.exists(nifti_file_path):
            print(f"[INFO] Using cached NIfTI: {nifti_file_path}")
            image = nib.load(nifti_file_path).get_fdata()
            images.append(image)
            labels.append(label)
            matched_records += 1
            continue  # Skip reprocessing

        matched = False
        for dicom_path in dicom_files:
            dicom = pydicom.dcmread(dicom_path)

            if dicom.SeriesNumber == series_num and series_desc in dicom.SeriesDescription:
                img = dicom.pixel_array.astype(np.float32)
                img = (img - np.min(img)) / (np.max(img) - np.min(img))  # Normalize
                img = np.expand_dims(img, axis=-1)  # Ensure 3D format (H, W, 1)
                img = tf.image.resize(img, img_size)  # Resize to (128, 128)

                images.append(img.numpy())
                labels.append(label)
                matched_records += 1
                matched = True

                # Save NIfTI in cache
                nifti_img = nib.Nifti1Image(img.numpy().squeeze(), np.eye(4))
                nib.save(nifti_img, nifti_file_path)
                print(f"[INFO] Saved NIfTI: {nifti_file_path}")

                break  # Stop checking once a match is found

        if not matched:
            print(f"[WARNING] No match found for SeriesNumber {series_num} and Description {series_desc}")

    print(f"[INFO] Total matched records: {matched_records}")
    return images, labels

# Convert/load images & labels
images, labels = load_or_convert_dicom_images(DATASET_PATH, merged_df)



[INFO] ImageTrain Columns: Index(['ProxID', 'Name', 'fid', 'pos', 'WorldMatrix', 'ijk', 'TopLevel',
       'SpacingBetweenSlices', 'VoxelSpacing', 'Dim', 'DCMSerDescr',
       'DCMSerNum'],
      dtype='object')
[INFO] FindingsTrain Columns: Index(['ProxID', 'fid', 'pos', 'zone', 'ClinSig'], dtype='object')
[INFO] Successfully merged! Total records: 187
Found 9988 DICOM files.
[INFO] Using cached NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0000.nii.gz
[INFO] Using cached NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0001.nii.gz
[INFO] Using cached NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0002.nii.gz
[INFO] Using cached NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0003.nii.gz
[INFO] Using cached NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0004.nii.gz
[INFO] Using cached NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0005.nii.gz
[INFO] Using cached NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cac

In [5]:
# Save images & labels as NIfTI in nnUNet format
for idx, (image, label) in enumerate(tqdm(zip(images, labels), desc="Saving NIfTI files", total=len(images))):
    image_nifti = nib.Nifti1Image(image.squeeze(), np.eye(4))
    nib.save(image_nifti, os.path.join(IMAGES_TR_PATH, f"prostate_{idx:04d}_0000.nii.gz"))

    #Convert `label` to an integer to fix dtype issue
    label_nifti = nib.Nifti1Image(np.array([[int(label)]], dtype=np.int16), np.eye(4))  # Convert to int16
    nib.save(label_nifti, os.path.join(LABELS_TR_PATH, f"prostate_{idx:04d}.nii.gz"))

Saving NIfTI files: 100%|██████████| 187/187 [00:05<00:00, 36.40it/s]


In [13]:

# Generate dataset.json
dataset_json = {
    "name": "ProstateX",
    "description": "ProstateX dataset for nnU-Net training",
    "tensorImageSize": "3D",
    "reference": "",
    "licence": "",
    "release": "1.0",
    "modality": {"0": "MRI"},
    "labels": {"0": "background", "1": "tumor"},
    "numTraining": len(images),
    "numTest": 0,
    "training": [{"image": f"./imagesTr/prostate_{idx:04d}_0000.nii.gz", "label": f"./labelsTr/prostate_{idx:04d}.nii.gz"} for idx in range(len(images))],
    "test": []
}

json_path = os.path.join(NNUNET_RAW_PATH, "dataset.json")
with open(json_path, "w") as f:
    json.dump(dataset_json, f, indent=4)

print("[INFO] Dataset preparation complete!")

# Step 4: Preprocess the Data
print("[INFO] Running nnUNet Preprocessing...")
os.system(f"nnUNetv2_plan_and_preprocess -d 500 --verify_dataset_integrity")

# Step 5: Train the Model
print("[INFO] Training nnUNet Model...")
os.system(f"nnUNetv2_train -d 500 -c 3d_fullres -f 0")

# Step 6: Inference
print("[INFO] Running Inference...")
os.system(f"nnUNetv2_predict -i {IMAGES_TR_PATH} -o /content/drive/MyDrive/nnUNet_predictions -d 500 -c 3d_fullres")



[INFO] Dataset preparation complete!
[INFO] Running nnUNet Preprocessing...
[INFO] Training nnUNet Model...
[INFO] Running Inference...


256

In [18]:
import os

os.environ["nnUNet_raw"] = "/content/drive/MyDrive/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/drive/MyDrive/nnUNet_results"

# Verify environment variables
print("[INFO] Environment Variables Set:")
print("nnUNet_raw:", os.environ.get("nnUNet_raw"))
print("nnUNet_preprocessed:", os.environ.get("nnUNet_preprocessed"))
print("nnUNet_results:", os.environ.get("nnUNet_results"))

[INFO] Environment Variables Set:
nnUNet_raw: /content/drive/MyDrive/nnUNet_raw
nnUNet_preprocessed: /content/drive/MyDrive/nnUNet_preprocessed
nnUNet_results: /content/drive/MyDrive/nnUNet_results


In [14]:
import subprocess

# Define paths
IMAGES_TR_PATH = "/content/drive/MyDrive/nnUNet_raw/Task500_ProstateX/imagesTr"
PREDICTION_PATH = "/content/drive/MyDrive/nnUNet_predictions"

# Ensure output directory exists
os.makedirs(PREDICTION_PATH, exist_ok=True)

# Run inference command and capture errors
inference_command = f"nnUNetv2_predict -i {IMAGES_TR_PATH} -o {PREDICTION_PATH} -d 500 -c 3d_fullres"
process = subprocess.run(inference_command, shell=True, capture_output=True, text=True)

# Print the command output
print("[INFO] Command Output:\n", process.stdout)
print("[ERROR] Command Error:\n", process.stderr)

# Check for failure
if process.returncode != 0:
    print(f"[ERROR] Inference failed with exit code {process.returncode}. Check the logs above.")


[INFO] Command Output:
 
#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


[ERROR] Command Error:
 Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_predict", line 8, in <module>
    sys.exit(predict_entry_point())
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/inference/predict_from_raw_data.py", line 948, in predict_entry_point
    model_folder = get_output_folder(args.d, args.tr, args.p, args.c)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/file_path_utilities.py", line 22, in get_outpu

In [15]:
DATASET_ID = "002"
inference_command = f"nnUNetv2_predict -i {IMAGES_TR_PATH} -o {PREDICTION_PATH} -d {DATASET_ID} -c 3d_fullres"


In [19]:
import subprocess

# Run inference command
inference_command = f"nnUNetv2_predict -i {IMAGES_TR_PATH} -o {PREDICTION_PATH} -d 500 -c 3d_fullres"
process = subprocess.run(inference_command, shell=True, capture_output=True, text=True)

# Print output and error
print("[INFO] Command Output:\n", process.stdout)
print("[ERROR] Command Error:\n", process.stderr)


[INFO] Command Output:
 
#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


[ERROR] Command Error:
 Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_predict", line 8, in <module>
    sys.exit(predict_entry_point())
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/inference/predict_from_raw_data.py", line 948, in predict_entry_point
    model_folder = get_output_folder(args.d, args.tr, args.p, args.c)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/file_path_utilities.py", line 22, in get_outpu

In [20]:
import os

nnUNet_preprocessed_path = "/content/drive/MyDrive/nnUNet_preprocessed"
nnUNet_results_path = "/content/drive/MyDrive/nnUNet_results"

# List available dataset folders
print("[INFO] Checking available datasets in nnUNet_preprocessed:")
print(os.listdir(nnUNet_preprocessed_path))

print("\n[INFO] Checking available trained models in nnUNet_results:")
print(os.listdir(nnUNet_results_path))


[INFO] Checking available datasets in nnUNet_preprocessed:


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/nnUNet_preprocessed'

In [21]:
import os

# Define paths
base_path = "/content/drive/MyDrive/"
paths = ["nnUNet_raw", "nnUNet_preprocessed", "nnUNet_results"]

for path in paths:
    full_path = os.path.join(base_path, path)
    print(f"[INFO] Checking path: {full_path}")
    print("Exists:", os.path.exists(full_path))
    print("Contents:", os.listdir(full_path) if os.path.exists(full_path) else "Folder not found")
    print("\n")



[INFO] Checking path: /content/drive/MyDrive/nnUNet_raw
Exists: True
Contents: ['Task500_ProstateX']


[INFO] Checking path: /content/drive/MyDrive/nnUNet_preprocessed
Exists: False
Contents: Folder not found


[INFO] Checking path: /content/drive/MyDrive/nnUNet_results
Exists: False
Contents: Folder not found




In [22]:
nnUNetv2_plan_and_preprocess -d 500 --verify_dataset_integrity


SyntaxError: invalid syntax (<ipython-input-22-5e82b98bdce9>, line 1)

In [16]:
import os
import shutil

# Define paths
TRAINED_MODEL_PATH = "/content/drive/MyDrive/nnUNet_results/nnUNet/3d_fullres/Task500_ProstateX"
BACKUP_MODEL_PATH = "/content/drive/MyDrive/nnUNet_trained_model_backup/Task500_ProstateX"

# Ensure backup directory exists
os.makedirs(os.path.dirname(BACKUP_MODEL_PATH), exist_ok=True)

# Copy the trained model to backup
if os.path.exists(TRAINED_MODEL_PATH):
    shutil.copytree(TRAINED_MODEL_PATH, BACKUP_MODEL_PATH, dirs_exist_ok=True)
    print(f"[INFO] Trained model successfully backed up to {BACKUP_MODEL_PATH}")
else:
    print("[ERROR] Trained model not found. Ensure training was completed successfully.")


[ERROR] Trained model not found. Ensure training was completed successfully.


In [23]:
import os
import subprocess

# Define paths
os.environ["nnUNet_raw"] = "/content/drive/MyDrive/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/drive/MyDrive/nnUNet_results"

# Verify environment variables
print("[INFO] Environment Variables Set:")
print("nnUNet_raw:", os.environ.get("nnUNet_raw"))
print("nnUNet_preprocessed:", os.environ.get("nnUNet_preprocessed"))
print("nnUNet_results:", os.environ.get("nnUNet_results"))

# Run nnU-Net Preprocessing
print("[INFO] Running nnU-Net Preprocessing...")
preprocess_command = "nnUNetv2_plan_and_preprocess -d 500 --verify_dataset_integrity"
process = subprocess.run(preprocess_command, shell=True, capture_output=True, text=True)

# Print command output
print("[INFO] Command Output:\n", process.stdout)
print("[ERROR] Command Error:\n", process.stderr)

# Verify if preprocessing was successful
preprocessed_path = "/content/drive/MyDrive/nnUNet_preprocessed/Task500_ProstateX"
if os.path.exists(preprocessed_path):
    print(f"[SUCCESS] Preprocessing completed! Preprocessed data is in: {preprocessed_path}")
    print("Listing preprocessed files:")
    print(os.listdir(preprocessed_path))
else:
    print("[ERROR] Preprocessing failed. Check the error logs above.")


[INFO] Environment Variables Set:
nnUNet_raw: /content/drive/MyDrive/nnUNet_raw
nnUNet_preprocessed: /content/drive/MyDrive/nnUNet_preprocessed
nnUNet_results: /content/drive/MyDrive/nnUNet_results
[INFO] Running nnU-Net Preprocessing...
[INFO] Command Output:
 Fingerprint extraction...

[ERROR] Command Error:
 Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_plan_and_preprocess", line 8, in <module>
    sys.exit(plan_and_preprocess_entry())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py", line 180, in plan_and_preprocess_entry
    extract_fingerprints(args.d, args.fpe, args.npfp, args.verify_dataset_integrity, args.clean, args.verbose)
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 47, in extract_fingerprints
    extract_fingerprint_dataset(d, fingerprint_extractor_class, num_processes, check_dataset